### Importazioni

In [22]:
from dotenv import dotenv_values
from pymongo import MongoClient
import json
import requests

## Estraggo le coordinate dai POI

In [23]:
# Apro il file e leggo il contenuto
with open('DICTIONARYTOTALE.txt', 'r') as file:
    contenuto = file.read()

In [24]:
# Decodifico il contenuto JSON in un dizionario Python
dizionario = json.loads(contenuto)

# Lista per le coordinate
coordinate = []

# Estraggo le longitudini e latitudini e le aggiungo alla lista coordinate
for elemento in dizionario.values():
    latitudine = elemento.get('lat')
    longitudine = elemento.get('lon')
    if latitudine is not None and longitudine is not None:
        coordinate.append((latitudine, longitudine))

# Stampo la lista delle coordinate
print(coordinate)

[(42.68827, 12.549764), (42.683333, 12.55), (42.690376, 12.54649), (42.691138, 12.54658), (42.69067, 12.546327), (42.70388888888889, 12.574444444444444), (42.698055555555555, 12.531388888888888), (42.673620035730664, 12.5873151), (42.673169, 12.603323), (42.656894, 12.536004), (42.7100669, 12.4951428), (42.70867, 12.48504), (42.664167, 12.508611), (42.71342, 12.50648), (42.711472, 12.611841), (42.708, 12.621), (42.65, 12.55), (42.64827, 12.637932), (42.73033, 12.58681), (42.7847, 12.0356), (42.786111, 12.037778), (42.772684, 12.039017), (42.771667, 12.018611), (42.804302, 12.071065), (42.75527777777778, 12.001666666666667), (42.80222222222222, 11.983888888888888), (42.833333, 12.066667), (42.835532, 12.065821), (42.816667, 11.966667), (42.755278, 11.945833), (42.7919, 11.9175), (42.7805, 11.9067), (42.587742, 12.295626), (42.583333, 12.3), (42.591907, 12.25983), (42.574862, 12.32824), (42.616667, 12.3), (42.566667, 12.333333), (42.62388, 12.29671), (42.621667, 12.254444), (42.546496, 1

In [25]:
print(str(len(coordinate)) + " POI")

3141 POI


In [26]:
poche_coordinate = coordinate[600:1000]

print(poche_coordinate)

[(42.440831, 12.44112), (42.440526, 12.44068), (42.481389, 12.568333), (42.49656, 12.49534), (42.5, 12.5), (42.45629, 12.59643), (42.465556, 12.586944), (42.476543, 12.621909), (42.449444, 12.659167), (42.484722, 12.652778), (42.478958, 12.442994), (42.48, 12.438056), (42.435883, 12.441331), (42.497735, 12.492468), (42.443056, 12.412222), (43.142887, 12.851864), (43.116667, 12.783333), (43.111044, 12.790139), (43.109919, 12.790779), (43.13, 12.86), (43.106104, 12.751745), (43.17955, 12.923630555555555), (43.188489, 12.931689), (43.130174, 12.922542), (43.1464, 12.949231), (43.1, 12.866667), (42.793333, 13.093889), (42.792333, 13.093389), (42.792208, 13.092377), (42.791775, 13.092753), (42.788253, 13.095972), (42.8, 13.1), (42.789202, 13.056521), (42.82033, 13.11648), (42.82449, 13.06506), (42.837358, 13.104186), (42.823822, 13.140369), (42.784928, 13.023628), (42.811667, 13.209444), (42.8289, 13.20582), (42.790607, 12.993017), (42.788333, 12.951667), (42.804442, 12.94881), (42.823108, 

In [27]:
print(len(poche_coordinate))

400


## Recupero la chiave API

In [28]:
dotenv = dotenv_values()
api_key = dotenv.get("API_KEY");

print(api_key)

8A20587981B148ADBBF3BED91ACFC937


## Estraggo i POI presenti su TripAdvisor

In [29]:
# Creo un insieme per memorizzare gli ID già presenti nel file
id_presenti = set()

# Controllo se il file esiste
import os
# Se il file esiste, leggo il contenuto attuale del file e memorizzo gli ID
if os.path.exists('poi.txt'):
    with open('poi.txt', 'r', encoding='latin-1') as file:
        for line in file:
            # Divido la riga usando ': ' come separatore solo alla prima occorrenza
            id, resto = line.strip().split(': ', 1)
            # Divido il resto della riga usando ', ' come separatore per ottenere il nome del POI
            nome = resto.split(', ')[0]
            id_presenti.add(id)
   
# Creo la lista vuota di POI da salvare
poi = []

# Mi salvo nella lista l'id e il nome del POI solo se l'id non è già presente
for coordinate in poche_coordinate:
    url = "https://api.content.tripadvisor.com/api/v1/location/nearby_search?latLong=" + str(coordinate[0]) + "%2C" + str(coordinate[1]) + "&key=" + api_key + "&category=attractions&language=it"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
   
    POI = response.json()

    # Estraggo l'id e il nome di ciascun POI e li aggiungo alla lista di POI se l'id non è già presente
    for elemento in POI['data']:
        nome = elemento.get('name')
        id = elemento.get('location_id')
        citta = elemento.get('address_obj')
        if nome is not None and id is not None and id not in id_presenti:
            poi.append((id, nome, citta))
            # Aggiungo l'id all'insieme degli ID presenti
            id_presenti.add(id)

# Apro il file in modalità scrittura in modalità append
with open('poi.txt', 'a', encoding='utf-8') as file:
    # Itero sulla lista di POI e scrivo ciascun POI nel file solo se l'id non è già presente
    for id, nome, citta in poi:
        file.write(f"{id}: {nome}, {citta}\n")

print(poi)
    

[('10804670', 'Chiesa dei Santi Vincenzo e Anastasio in Finocchieto', {'street2': 'Località Finocchieto, 2', 'city': 'Stroncone', 'state': 'Provincia di Terni', 'country': 'Italia', 'address_string': 'Località Finocchieto, 2, Stroncone Italia'}), ('18716739', 'Santuario del Beato Antonio Vici', {'street1': 'Via San Francesco 24', 'street2': '', 'city': 'Stroncone', 'state': 'Provincia di Terni', 'country': 'Italia', 'postalcode': '05039', 'address_string': 'Via San Francesco 24, 05039 Stroncone Italia'}), ('10804668', 'Palazzo Comunale', {'street1': "Piazza della Liberta' 6", 'street2': '', 'city': 'Stroncone', 'state': 'Provincia di Terni', 'country': 'Italia', 'postalcode': '05039', 'address_string': "Piazza della Liberta' 6, 05039 Stroncone Italia"}), ('23696050', 'Chiesa San Giovanni Decollato', {'street1': 'Piazza S.giovanni', 'street2': '', 'city': 'Stroncone', 'state': 'Provincia di Terni', 'country': 'Italia', 'postalcode': '05039', 'address_string': 'Piazza S.giovanni, 05039 S

In [30]:
print(len(poi))

564


In [31]:
# Inizializzo un dizionario vuoto per memorizzare gli ID e i nomi dei POI
poi_dict = {}

# Leggo il contenuto del file riga per riga
with open('poi.txt', 'r', encoding='latin-1') as file:
    for line in file:
        # Divido la riga usando ': ' come separatore solo alla prima occorrenza
        id, resto = line.strip().split(': ', 1)
        # Divido il resto della riga usando ', ' come separatore per ottenere il nome del POI
        nome = resto.split(', ')[0]
        # Aggiungo l'id e il nome al dizionario
        poi_dict[id] = nome

# Ora poi_dict conterrà gli ID come chiavi e i nomi dei POI come valori
print(poi_dict)


{'8666261': 'Palazzo Cesi', '13436625': 'Chiesa di Santa Cecilia', '21223808': 'Lago Di Arezzo', '19071050': 'Frantoio Oleario Palmadori Massimo', '2016179': 'Carsulae - Parco Archeologico', '19487544': 'Chiesa San Bernardino', '19485030': 'Chiesa di Santï¿½Apollinare', '21389509': 'Chiesa San Faustino', '17785314': "Pieve di Sant'Antonio Abate", '17780645': "Chiesa di Sant'Antonio Abate", '17781582': "Chiesa Di Sant'antonio Abate", '617225': "Velia's Cooking Style", '11851556': 'Bottega del Castello della Sale', '10384019': 'Cirulli - Aziende Agricole', '12673247': 'Castello di Madonna Antonia', '17780814': 'chiesa della S.S. Annunziata', '27514189': 'Parrucchiera Giulia', '13436641': 'Museo dei Cicli Geologici', '13436639': 'Chiesa di San Michele Arcangelo', '13436560': "Chiesa della Madonna dell'Acqua", '4828006': 'Borgo Antico Bar', '10727071': 'Chiesa Santa Maria Assunta', '26437754': 'Shiatsuqi di Natascia Minciarelli', '26628676': 'Le Fontane - Lavatoio pubblico', '17780615': 'A

In [3]:
for elemento in poi_dict:
    print(elemento)

8666261
13436625
21223808
19071050
2016179
19487544
19485030
21389509
17785314
17780645
17781582
617225
11851556
10384019
12673247
17780814
27514189
13436641
13436639
13436560
4828006
10727071
26437754
26628676
17780615
23603031
26454141
23498616
17785149
23527414
17784926
8442796
6362879
3316846
10751519
6873086
13506803
23736060
5988820
13908621
8818259
15234702
19486021
19485904
21129953
15234713
7927400
19635998
12827750
5937150
13908574
26858943
26848269
10717566
23590891
24077147
21129964
19485106
15582334
15582326
12941174
15582304
12941300
20943988
4178733
15582352
9974289
14902761
10536774
15582310
17567725
10125575
21330794
21341165
23082361
21341156
23082363
15582323
21341167
21341166
4103928
20947702
10120122
7117226
1583158
23919722
21272422
23066524
23430908
21330795
21329540
4131906
23410936
19485109
15843151
10821335
6878472
12819225
10820447
10820435
10820353
19940269
19344024
10342709
23682565
25336273
24033005
3400132
1392223
20724089
20724098
11502819
13436659
66091

In [33]:
len(poi_dict)

1609

### Salvo i POI in un dizionario JSON

In [34]:
# salvo il dizionario in un file json
dizionario_json = "poi.json"

# Salvataggio del dizionario nel file JSON
with open(dizionario_json, "w") as file:
    json.dump(poi_dict, file)

## Recupero le recensioni relative ai POI estratti

In [35]:
items_list = list(poi_dict.items())
print(items_list)

for elemento in items_list[713:]:
    print(elemento[0])


[('8666261', 'Palazzo Cesi'), ('13436625', 'Chiesa di Santa Cecilia'), ('21223808', 'Lago Di Arezzo'), ('19071050', 'Frantoio Oleario Palmadori Massimo'), ('2016179', 'Carsulae - Parco Archeologico'), ('19487544', 'Chiesa San Bernardino'), ('19485030', 'Chiesa di Santï¿½Apollinare'), ('21389509', 'Chiesa San Faustino'), ('17785314', "Pieve di Sant'Antonio Abate"), ('17780645', "Chiesa di Sant'Antonio Abate"), ('17781582', "Chiesa Di Sant'antonio Abate"), ('617225', "Velia's Cooking Style"), ('11851556', 'Bottega del Castello della Sale'), ('10384019', 'Cirulli - Aziende Agricole'), ('12673247', 'Castello di Madonna Antonia'), ('17780814', 'chiesa della S.S. Annunziata'), ('27514189', 'Parrucchiera Giulia'), ('13436641', 'Museo dei Cicli Geologici'), ('13436639', 'Chiesa di San Michele Arcangelo'), ('13436560', "Chiesa della Madonna dell'Acqua"), ('4828006', 'Borgo Antico Bar'), ('10727071', 'Chiesa Santa Maria Assunta'), ('26437754', 'Shiatsuqi di Natascia Minciarelli'), ('26628676', '

In [45]:
# Dizionario per memorizzare le recensioni per ogni location
recensioni_per_location = {}

items_list = list(poi_dict.items())
# Itero sui POI e recupera le recensioni
for elemento in items_list[713:]:
    url = "https://api.content.tripadvisor.com/api/v1/location/" + elemento[0] + "/reviews?key=" + api_key + "&language=it"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    REC = response.json()

    # Estraggo le recensioni di ciascun POI e le aggiungo al dizionario
    for elemento in REC['data']:
        id_rec = elemento.get('id')
        id_loc = elemento.get('location_id')
        rating = elemento.get('rating')
        recensione = elemento.get('text')

        # Se la location_id non è già presente nel dizionario, l'aggiungo come chiave con una lista vuota come valore
        if id_loc not in recensioni_per_location:
            recensioni_per_location[id_loc] = {}

        # Aggiungo la recensione al dizionario nidificato utilizzando l'id della recensione come chiave
        recensioni_per_location[id_loc][id_rec] = {'rating': rating, 'recensione': recensione}

# Scrivo il dizionario di recensioni su un file JSON
with open('recensioni_per_location1.json', 'w') as file:
    json.dump(recensioni_per_location, file)

In [46]:
REC

{'data': [], 'paging': {}}

In [47]:
len(recensioni_per_location)

248

In [54]:
# Apro il file in modalità lettura
with open('recensioni_complete.json', 'r') as file:
    recensioni = file.read();
    # Decodifico il contenuto JSON in un dizionario Python

print(recensioni)
print(len(recensioni))
dizionario_recensioni = json.loads(recensioni)

{
    "8666261": {
        "POI": "Palazzo Cesi",
        "917883573": {
            "rating": 4,
            "recensione": "Palazzo Cesi di Acquasparta \u00e9 il principale monumento del borgo.\nVisitabile con un esiguo biglietto di 6 euro, venne fondato dal Duca Federico Cesi nel Seicento, e nelle sue sale venne ospitato Galileo Galilei.\nIl prospetto esterno, che d\u00e0 sulla piazza principale, si presenta, nell\u2019ordine inferiore, con uno splendido portale rinascimentale a tutto sesto, ornato da una cornice in conci bugnati.\nL\u2019interno \u00e9 impreziosito da un susseguirsi di sale che presentano splendide volte affrescate, in eccellente stato di conservazione, mentre altre sono ornate da elaborati soffitti lignei. Gli affreschi presenti, narrano vicende legate alla mitologia.\nLa struttura del palazzo \u00e9 edificata su due piani, piano terra e primo piano, entrambi visitabili. Splendido \u00e9 anche il loggiato del primo piano, ove si affacciano le stanze. Le varie sale 

In [55]:
# recupero anche il dizionario con solo chiave univoca della location e il nome
with open('poi.json', 'r') as file:
    poi_json = file.read()
    dizionario_poi = json.loads(poi_json)

In [56]:
for items in dizionario_poi.items():
    for idloc, recensione_info in dizionario_recensioni.items():
        recensione_info = {'POI': "nome_poi", **recensione_info}

In [57]:
for id_rec, recensione_info in dizionario_recensioni.items():
    # Controllo se l'id della recensione corrisponde a un id del POI
    if id_rec in dizionario_poi:
        # Ottengo il nome del POI associato all'id del POI
        nome_poi = dizionario_poi[id_rec]
        # Aggiungo il nome del POI all'inizio della recensione
        recensione_info = {'POI': nome_poi, **recensione_info}
        # Aggiorno il dizionario delle recensioni con la recensione aggiornata
        dizionario_recensioni[id_rec] = recensione_info

# Ora dizionario_recensioni contiene il campo 'POI' aggiunto all'inizio di ciascuna recensione



In [58]:
# Scrivo il dizionario di recensioni su un file JSON
with open('recensioni_complete.json', 'w') as file:
    json.dump(dizionario_recensioni, file)

In [59]:
# Conto le recensioni estratte per ora

with open('recensioni_complete.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inizializzo il contatore delle recensioni
numero_recensioni = 0

# Itero attraverso i dati ed estraggo le recensioni
for poi_id, poi_data in data.items():
    for recensione_id, recensione_info in poi_data.items():
        if isinstance(recensione_info, dict) and 'recensione' in recensione_info:
            recensione = recensione_info['recensione']
            print(recensione)  # Stampo la recensione 
            numero_recensioni += 1

# Stampa il numero totale di recensioni
print("Numero totale di recensioni:", numero_recensioni)


Palazzo Cesi di Acquasparta é il principale monumento del borgo.
Visitabile con un esiguo biglietto di 6 euro, venne fondato dal Duca Federico Cesi nel Seicento, e nelle sue sale venne ospitato Galileo Galilei.
Il prospetto esterno, che dà sulla piazza principale, si presenta, nell’ordine inferiore, con uno splendido portale rinascimentale a tutto sesto, ornato da una cornice in conci bugnati.
L’interno é impreziosito da un susseguirsi di sale che presentano splendide volte affrescate, in eccellente stato di conservazione, mentre altre sono ornate da elaborati soffitti lignei. Gli affreschi presenti, narrano vicende legate alla mitologia.
La struttura del palazzo é edificata su due piani, piano terra e primo piano, entrambi visitabili. Splendido é anche il loggiato del primo piano, ove si affacciano le stanze. Le varie sale che si susseguono nel percorso di visita sono  consequenziali tra loro, sia al piano terra che al primo, pertanto non c’é alcun rischio di perdersi nel palazzo o di

# Salvo nel database noSQL (MongoDB)


In [12]:
# Connessione al server MongoDB
client = MongoClient('mongodb://localhost:27017/')

In [13]:
# Seleziona il database
db = client['RASTA']

# Seleziona la collezione
collection = db['poi']

In [14]:

# Caricamento dei dati dal file JSON
with open('recensioni_complete.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

for key, value in data.items():
    db.collection_name.insert_one(value)

# Chiusura della connessione al server MongoDB
client.close()

In [21]:
# Connessione al server MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Seleziona il database
db = client['RASTA']

# Seleziona la collezione
collection = db['poi']

# Query per trovare tutte le recensioni 
cursor = collection.find()

# Iterazione sui risultati della query
for poi in cursor:
    # Accesso al nome del POI
    poi_name = poi['POI']
    
    # Accesso alle recensioni associate al POI
    reviews = poi.values()
    
    # Iterazione sulle recensioni
    for review in reviews:
        # Controllo se l'elemento è un dizionario (ovvero una recensione)
        if isinstance(review, dict):
            # Accesso al rating della recensione
            rating = review['rating']
            print(f"POI: {poi_name}, Rating: {rating}")
    
# Chiusura della connessione al server MongoDB
client.close()

POI: Palazzo Cesi, Rating: 4
POI: Palazzo Cesi, Rating: 4
POI: Palazzo Cesi, Rating: 4
POI: Palazzo Cesi, Rating: 5
POI: Palazzo Cesi, Rating: 5
POI: Chiesa di Santa Cecilia, Rating: 3
POI: Chiesa di Santa Cecilia, Rating: 5
POI: Chiesa di Santa Cecilia, Rating: 3
POI: Lago Di Arezzo, Rating: 4
POI: Frantoio Oleario Palmadori Massimo, Rating: 4
POI: Frantoio Oleario Palmadori Massimo, Rating: 5
POI: Frantoio Oleario Palmadori Massimo, Rating: 5
POI: Frantoio Oleario Palmadori Massimo, Rating: 5
POI: Carsulae - Parco Archeologico, Rating: 5
POI: Carsulae - Parco Archeologico, Rating: 5
POI: Carsulae - Parco Archeologico, Rating: 4
POI: Carsulae - Parco Archeologico, Rating: 5
POI: Carsulae - Parco Archeologico, Rating: 5
POI: Pieve di Sant'Antonio Abate, Rating: 5
POI: Chiesa di Sant'Antonio Abate, Rating: 5
POI: Chiesa di Sant'Antonio Abate, Rating: 5
POI: Chiesa Di Sant'antonio Abate, Rating: 4
POI: Chiesa Di Sant'antonio Abate, Rating: 5
POI: Velia's Cooking Style, Rating: 5
POI: Bot